# FashionMnist with confusion matrix

In [1]:
# imports
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [5]:
device_count = torch.cuda.device_count()
if device_count > 0:
    print(f"Number of CUDA devices available: {device_count}")
    for device_idx in range(device_count):
        device_name = torch.cuda.get_device_name(device_idx)
        print(f"Device {device_idx}: {device_name}")
else:
    print("No CUDA devices found.")
    
# Specify the device type
torch.cuda.is_available()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Number of CUDA devices available: 1
Device 0: AMD Radeon Pro W6800


device(type='cuda')

In [2]:
# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# batch_size
batch_size = 128

# datasets
trainset = torchvision.datasets.FashionMNIST('../data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('../data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

100.0%


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100.0%


Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



100.0%


Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100.0%

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

net.to(device)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [8]:
running_loss = 0.0
accuracy = 0.0
epochs = 10
for epoch in range(epochs):  # loop over the dataset multiple times
    print('Epoch-{0} lr: {1}'.format(epoch + 1, optimizer.param_groups[0]['lr']))
    for i, data in enumerate(trainloader, 0):
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:    # every 1000 mini-batches...

            # ...log the running loss
            print("Training loss {} Steps: {}".format(running_loss / 1000, epoch * len(trainloader) + i))
            running_loss = 0.0
print('Finished Training')


Epoch-1 lr: 0.001


MIOpen(HIP): Warning [SQLiteBase] Missing system database file: gfx1030_30.kdb Performance may degrade. Please follow instructions to install: https://github.com/ROCmSoftwarePlatform/MIOpen#installing-miopen-kernels-package


Epoch-2 lr: 0.001
Epoch-3 lr: 0.001
Epoch-4 lr: 0.001
Epoch-5 lr: 0.001
Epoch-6 lr: 0.001
Epoch-7 lr: 0.001
Epoch-8 lr: 0.001
Epoch-9 lr: 0.001
Epoch-10 lr: 0.001
Finished Training


In [10]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

y_pred = []
y_true = []

# iterate over test data
for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        output = net(inputs) # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) # Save Prediction
        
        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth

        # print(f'output:\n{output}')
        # print(f'labels:\n{labels}')

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,10))
#plt.title('Confusion Matrix')
s = sn.heatmap(df_cm, cmap='Blues', fmt='.2f', square=True, annot=True)
s.set_xlabel('True', fontsize=14)
s.set_ylabel('Predict', fontsize=14)
plt.savefig('../imgnet_conf_matrix.png')

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [28]:
classes =  ['dog', 'cat', 'bird', 'fish']
samples = np.arange(0, len(classes))
o_true = np.random.choice(samples, size=10)
print(f'true: {o_true}')
o_pred = np.random.choice(samples, size=10)
print(f'pred: {o_pred}')

cm = confusion_matrix(o_true, o_pred)
print(cm)

true: [2 2 1 3 3 0 2 1 3 2]
pred: [2 1 3 1 0 0 1 2 2 2]
[[1 0 0 0]
 [0 0 1 1]
 [0 2 2 0]
 [1 1 1 0]]
